In [21]:
from langchain_groq import ChatGroq
import os

In [22]:
llm = ChatGroq(
    temperature=0.7, 
    groq_api_key=os.getenv("LLaMa_API_KEY"), 
    model_name="llama-3.3-70b-versatile"
)

In [23]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/career-areas")
page_data = loader.load().pop().page_content
print(page_data)





















Career Areas








































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Languag

In [24]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [25]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'Retail Associate',
  'experience': 'Entry-level',
  'skills': '',
  'description': 'Work in a retail environment, sharing passion for products and helping customers unlock their inner athlete.'},
 {'role': 'Retail Assistant (Athlete)',
  'experience': 'Part-time',
  'skills': '',
  'description': 'Assist customers, share product expertise and help others unlock their inner athlete.'},
 {'role': 'Director, Software Engineering',
  'experience': 'Senior-level',
  'skills': 'Data, Analytics, Intelligence',
  'description': 'Lead software engineering team, focusing on data, analytics, and intelligence.'},
 {'role': 'Director, Software Engineering',
  'experience': 'Senior-level',
  'skills': 'Data, Analytics, Intelligence',
  'description': 'Lead software engineering team, focusing on data, analytics, and intelligence.'},
 {'role': 'Senior Director, Software Engineering',
  'experience': 'Executive-level',
  'skills': 'Data, Analytics, Intelligence',
  'description': 'Lead softw

In [26]:
import json
print(res.content)

if res.content.strip().startswith('['):
    parsed_content = json.loads(res.content)
    
    if parsed_content:
        json_res = parsed_content[0]
    else:
        json_res = {}
elif res.content.strip().startswith('{'):
    json_res = json.loads(res.content)
else:
    json_res = {}
print(json_res)
print(type(json_res))

```json
[
  {
    "role": "Retail Associate",
    "experience": "Entry-level",
    "skills": "",
    "description": "Work in a retail environment, sharing passion for products and helping customers unlock their inner athlete."
  },
  {
    "role": "Retail Assistant (Athlete)",
    "experience": "Part-time",
    "skills": "",
    "description": "Assist customers, share product expertise and help others unlock their inner athlete."
  },
  {
    "role": "Director, Software Engineering",
    "experience": "Senior-level",
    "skills": "Data, Analytics, Intelligence",
    "description": "Lead software engineering team, focusing on data, analytics, and intelligence."
  },
  {
    "role": "Director, Software Engineering",
    "experience": "Senior-level",
    "skills": "Data, Analytics, Intelligence",
    "description": "Lead software engineering team, focusing on data, analytics, and intelligence."
  },
  {
    "role": "Senior Director, Software Engineering",
    "experience": "Executive-lev

In [27]:
type(json_res)

dict

In [28]:
import pandas as pd

df = pd.read_csv("App/Resources/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [29]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [30]:
links = collection.query(query_texts=["skills"], n_results=2).get('metadatas', [])
links

[[{'links': 'https://nayeemhossenjim.github.io/Portfolio/'},
  {'links': 'https://example.com/python-portfolio'}]]

In [31]:
job = json_res
skills = job.get('skills')
print(skills)


None


In [32]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Nayeem Hossen Jim, a passionate ML engineer and game developer. Your role is to craft a professional cold email to the client regarding the job mentioned above. In the email, highlight AtliQ’s expertise in meeting their business needs, while emphasizing the value we bring through our AI-driven solutions and software consulting services.

        The email should reflect AtliQ’s proven track record in empowering businesses by integrating advanced technologies, optimizing processes, and delivering customized automation solutions that ensure scalability, cost efficiency, and enhanced productivity. Make sure to showcase how our tailored solutions have helped other businesses achieve remarkable success.

        Additionally, you should incorporate relevant examples from the portfolio links provided below, emphasizing our experience and showcasing our successful projects in areas that directly align with the client’s requirements.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)


Subject: Unlocking Business Potential with AI-Driven Solutions and Software Consulting

Dear Hiring Manager,

I came across the job description and was impressed by the opportunity to collaborate with your team. As a passionate ML engineer and game developer at AtliQ, I am excited to introduce our expertise in meeting your business needs through cutting-edge AI-driven solutions and software consulting services.

At AtliQ, we have a proven track record of empowering businesses by integrating advanced technologies, optimizing processes, and delivering customized automation solutions that ensure scalability, cost efficiency, and enhanced productivity. Our tailored solutions have helped numerous businesses achieve remarkable success, and we are confident that we can do the same for your organization.

Our expertise in AI and machine learning has enabled us to develop innovative solutions that drive business growth and improvement. For instance, our work in natural language processing has h